In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 1500)

In [ ]:
df = pd.read_csv("all.csv")
df = df[["NAZEV_DOPLNEK"]]
df = df[~df["NAZEV_DOPLNEK"].isnull()]

In [ ]:
df = df.sort_values(by="NAZEV_DOPLNEK").reset_index(drop=True)

In [ ]:
import re

In [ ]:
volume_units = {"ML", "DL", "L"}


def parser(label):
    def match_simple(s):
        """
        Parse a name containing a separate number (without units), interpreted as the
        amount, and a number followed by a unit, interpreted as the amount in package.
        """
        # 1 group - amount
        regex_amount = re.compile(r"((?:\d*\.)?\d+)")
        
        # 3 groups - entire match,  in_package, in_package_units
        regex_in_package = re.compile(r"(((?:\d*\.)?\d+) ?(MG|G|MCG|KU|IU|IR|GM|RG))")
        
        # -> apply together
        match_in_package = regex_in_package.match(s)
        
        if match_in_package is None:
            return None
        
        in_package_total_match = match_in_package.groups()[0]
        in_package = match_in_package.groups()[1]
        in_package_units = match_in_package.groups()[2]
        
        match_amount = regex_amount.search(s.replace(in_package_total_match, ""))
        
        if match_amount is None:
            return None
        
        amount = match_amount.groups()[0]
        
        return {
            "amount": amount,
            "in_package": in_package,
            "in_package_units": in_package_units,
            "_source": "simple",
        }
        
    def match_complex1(s):
        """
        Parse a string containing either a string such as 1X30VOLUME_UNIT,
        interpreted as the amount and its units, or a string such as
        3X50WEIGHT_UNIT, interpreted as the amount, amount in package and
        its units.
        """
        # 2 groups - amount, amount_units
        regex_amount = re.compile(r"((?:(?:\d*\.)?\d+X)*(?:\d*\.)?\d+) ?(L|ML)?")

        # 3 groups - amount, in_package, in_package_units
        regex_amount_in_package = re.compile(
            r"((?:(?:\d*\.)?\d+X)*(?:\d*\.)?\d+)X((?:\d*\.)?\d+) ?(MG|G|MCG|KU|IU|IR|GM|RG)"
        )
        
        # -> apply one or the other
        match_amount = regex_amount.match(s)
        
        if match_amount is not None:
            amount = match_amount.groups()[0]
            amount_units = match_amount.groups()[1]
            
            return {
                "amount": amount,
                "amount_units": amount_units,
                "_source": "complex1",
            }
        
        match_amount_in_package = regex_amount_in_package.match(s)
        
        if match_amount_in_package is not None:
            amount = match_amount_in_package.groups()[0]
            in_package = match_amount_in_package.groups()[1]
            in_package_units = match_amount_in_package.groups()[2]
            
            return{
                "amount": amount,
                "in_package": in_package,
                "in_package_units": in_package_units,
                "_source": "complex1",
            }
        
        return None
        
    def match_complex2(s):
        """
        Parse a name containing a string such as 1X40 or 3X28, interpreted as the amount, and
        1MG/3MG or 0.5MG/ML, interpreted as the amount in package.
        """
        # 1 group - in_package
        regex_in_package = re.compile(r"((?:\d*\.)?\d+ ?[A-Z]+(?:/(?:(?:\d*\.)?\d+)? ?[A-Z]+)*)")
        match_in_package = regex_in_package.match(s)
        
        if match_in_package is None:
            return None
        
        in_package = match_in_package.groups()[0]
        
        # 2 groups - amount, amount_units
        regex_amount = re.compile(r"((?:(?:\d*\.)?\d+X)*(?:\d*\.)?\d+) ?([A-Z]*)")
        
        match_amount = regex_amount.search(s.replace(in_package, ""))
        
        if match_amount is None:
            return None
        
        amount = match_amount.groups()[0]
        amount_units = match_amount.groups()[1]
        
        return {
            "amount": amount,
            "amount_units": amount_units,
            "in_package": in_package,
            "_source": "complex2",
        }
    
    def match_units_in_x_part(s):
        """
        Parse a name containing a string such as 1X50MG or 10X500ML.
        """    
        regex = re.compile(r"((?:\d*\.)?\d+)X((?:\d*\.)?\d+) ?([A-Z]+)")
        match = regex.search(s)

        if match is None:
            return None

        num1, num2, units = match.groups()

        if units in volume_units:
            return {
                "amount": num1 + "X" + num2,
                "amount_units": units,
                "_source": "units_in_x_part",
            }
        else:
            return {
                "amount": num1,
                "in_package": num2,
                "in_package_units": units,
                "_source": "units_in_x_part",
            }

    # Replace decimal commas by points.
    
    if pd.isnull(label):
        row = None
    else:
        label = label.replace(",", ".")

        # 1. Try to match the most complex pattern.
        row = match_complex2(label)

        # 2. Try to match a simpler pattern.
        if row is None:
            row = match_complex1(label)

        # 3. Try to match the simplest pattern.
        if row is None:
            row = match_simple(label)

        # 4. Try to match for units appearing in the X part.
        if row is None:
            row = match_units_in_x_part(label)
    
    # 5. Unable to parse the row.
    if row is None:    
        row = {"original": label, "is_ok": False}
    else:
        row["original"] = label
        row["is_ok"] = True

    index = ["original", "is_ok", "amount", "amount_units", "in_package", "in_package_units", "_source"]
    return pd.Series(row, index=index)

In [ ]:
# Match string in row["NAZEV_DOPLNEK"] containing "+" and replace it with empty string

def plus(row):
    label = row["NAZEV_DOPLNEK"]
    row["SUPER_ORIGINAL"] = label
    
    # Match (num+num).
    regex_num = re.compile(r"(\((?:\d*\.)?\d+ ?\+ ?(?:\d*\.)?\d+\))")
    if regex_num.search(label):
        row["NAZEV_DOPLNEK"] = regex_num.sub("", row["NAZEV_DOPLNEK"]).strip()
        return row
    
    # Match +numJ.
    regex_numJ = re.compile(r"(\+ ?(?:\d*\.)?\d+J)")
    if regex_numJ.search(label):
        row["NAZEV_DOPLNEK"] = regex_numJ.sub("", row["NAZEV_DOPLNEK"]).strip()
        return row
    
    # Match +numINH.
    regex_numINH = re.compile(r"(\+ ?(?:\d*\.)?\d+INH)")
    if regex_numINH.search(label):
        row["NAZEV_DOPLNEK"] = regex_numINH.sub("", row["NAZEV_DOPLNEK"]).strip()
        return row
    
    # Match +string.
    regex_string = re.compile(r"(\+\w+)")
    if regex_string.search(label):
        row["NAZEV_DOPLNEK"] = regex_string.sub("", row["NAZEV_DOPLNEK"]).strip()
        return row
    
    # Match string+string.
    regex_str_str = re.compile(r"(\w+\+\w+)")
    if regex_str_str.search(label):
        row["NAZEV_DOPLNEK"] = regex_str_str.sub("", row["NAZEV_DOPLNEK"]).strip()
        return row

    return row

In [ ]:
df[df["NAZEV_DOPLNEK"].str.contains("\+")].head(10)

In [ ]:
masked_df = df.apply(plus, axis=1)
masked_df

In [ ]:
plus_df = masked_df[masked_df["NAZEV_DOPLNEK"].str.contains(r"\+")]
df = masked_df[~masked_df["NAZEV_DOPLNEK"].str.contains(r"\+")]

In [ ]:
not_ok4 = plus_df
not_ok4

In [ ]:
parsed = df["NAZEV_DOPLNEK"].apply(parser)
parsed["SUPER_ORIGINAL"] = df["SUPER_ORIGINAL"]

In [ ]:
parsed.head(100)

In [ ]:
parsed[~parsed["is_ok"]].head(30)

In [ ]:
def product(values):
    result = 1.0
    
    for value in values:
        result *= value
        
    return result


def handle_nested_value(row):
    """
    Match amount of the form 2X3 and in_package of the form 1MG/3MG or 4MG/5ML.
    The amount is always multiplied together (i.e., 2X3 -> 6), the in_package part
    depends on the units. All values appearing with non-volume units are added together,
    upon encountering any value paired with a volume unit, the subresult is divided by
    that value.
    """
    amount, in_package = row["amount"], row["in_package"]
    amount_regex = re.compile(r"(?:((?:\d*\.)?\d+)X)*((?:\d*\.)?\d+)")
    in_package_regex = re.compile(r"((?:\d*\.)?\d+)? ?([A-Z]+)")
    
    amount_match = amount_regex.fullmatch(amount)
    
    if not amount_match:
        return None
    
    in_package_vals = []
    in_package_units = []
    
    for subpart in in_package.split("/"):
        subpart_match = in_package_regex.fullmatch(subpart)
        
        if not subpart_match:
            return None
        
        val, unit = subpart_match.groups()
        in_package_vals.append(val)
        in_package_units.append(unit)

    amounts = [float(am) for am in amount_match.groups() if am is not None]
    new_amount = product(amounts)
    
    new_in_package = 0.0
    
    for value, unit in zip(in_package_vals, in_package_units):
        if unit in volume_units:
            new_in_package /= float(value) if value is not None else 1.0
        else:
            if value is None:
                return None
            
            new_in_package += float(value)

    row["processed_amount"] = new_amount
    row["processed_in_package"] = new_in_package
    row["in_package_units"] = in_package_units[0]
    
    return row


def process_amounts_and_units(row):
    amount, in_package = row["amount"], row["in_package"]
    
    if amount == "" or pd.isnull(amount) or in_package == "" or pd.isnull(in_package) or not row["is_ok"]:
        row["is_ok"] = False
        row["_postprocessing"] = "none"
        return row
    else:
        regex = re.compile(r"(\d+) ?([A-Z]+)")
        match = regex.fullmatch(in_package)
        
        if not match or "X" in amount:
            nested = handle_nested_value(row)

            if nested is None:
                row["is_ok"] = False
                row["_postprocessing"] = "nested_failed"
                return row
            
            nested["_postprocessing"] = "nested"
            return nested
        
        value, unit = match.groups()
        
        amount = float(amount)
        value = float(value)
        
        if unit in volume_units:
            new_amount = amount * value
            new_in_package = float("nan")
        else:
            new_amount = amount
            new_in_package = value
        
        row["processed_amount"] = new_amount
        row["processed_in_package"] = new_in_package
        row["in_package_units"] = unit
        row["_postprocessing"] = "simple"
        
        return row


In [ ]:
parsed_processed = parsed.apply(process_amounts_and_units, axis=1)
parsed_processed = parsed_processed[[
    "original",
    "SUPER_ORIGINAL",
    "amount",
    "in_package",
    "_source",
    "processed_amount",
    "amount_units",
    "processed_in_package",
    "in_package_units",
    "_postprocessing",
    "is_ok",
]]

In [ ]:
parsed_processed.head(100)

In [ ]:
volume_units_present = parsed_processed["amount_units"] == "ML"
volume = parsed_processed[volume_units_present]#.reset_index(drop=True)

non_volume = parsed_processed[~volume_units_present]#.reset_index(drop=True)
alright = non_volume[non_volume["is_ok"]]
to_check = non_volume[~non_volume["is_ok"]]


In [ ]:
volume

In [ ]:
to_check

In [ ]:
#If there is correct amount and in_package in to_check table, then:
def correction(row):
    amount, in_package = row["amount"], row["in_package"]
    
    if amount == "" or pd.isnull(amount) or in_package == "" or pd.isnull(in_package):
        row["is_ok"] = False
        return row
    elif row["amount_units"] == "D" or row["amount_units"] == "L":
        row["is_ok"] = False
        return row
    elif row["_source"] == "complex1" or row["_source"] == "complex2":
        row["is_ok"] = False
        return row
    elif row["in_package_units"] == "D" or row["in_package_units"] == "X" or row["in_package_units"] =="LTX" or row["in_package_units"] == "GMX" or row["in_package_units"] == "LT" or row["in_package_units"] == "MLX" or row["in_package_units"] == "MGX" or row["in_package_units"] == "LAH" or row["in_package_units"] == "MLPELAH" or row["in_package_units"] == "MLSKLO" or row["in_package_units"] == "MLPE" or row["in_package_units"] == "GB":
        row["is_ok"] = False
        return row
    else:
        amount = float(amount)
        in_package = float(in_package)
        row["processed_amount"] = amount
        row["processed_in_package"] = in_package
        row["is_ok"] = True
        return row

In [ ]:
checked = to_check.apply(correction, axis = 1)
checked

In [ ]:
alright["amount_in_package"] = alright["processed_amount"] * alright["processed_in_package"]

In [ ]:
alright

In [ ]:
alright2 = checked[checked["is_ok"]]
alright2["amount_in_package"] = alright2["processed_amount"] * alright2["processed_in_package"]
alright2

In [ ]:
alright = pd.concat([alright,alright2] , axis = "index").reset_index(drop=True)
alright

In [ ]:
alright3 = volume[volume["is_ok"]]
alright3["amount_in_package"] = alright3["processed_amount"] * alright3["processed_in_package"]
alright3

In [ ]:
alright = pd.concat([alright, alright3], axis = "index").reset_index(drop=True)
alright

In [ ]:
def unification(row):
    amount_in_pckg = row["amount_in_package"]
    
    """
    For further calculations unify in_package_units to grams "G" if unit is in "MG" or "MCG"
    """
    
    if row["in_package_units"] == "MG":
        row["amount_in_package_g"] = amount_in_pckg / 1000
        return row
    elif row["in_package_units"] == "MCG":
        row["amount_in_package_g"] = amount_in_pckg / 1000000
        return row
    else:
        return row
        

In [ ]:
new_alright = alright.apply(unification, axis = 1)
new_alright

In [ ]:
not_ok1 = volume[~volume["is_ok"]]
not_ok2 = pokus[~pokus["is_ok"]]
not_ok2

In [ ]:
not_ok = pd.concat([not_ok1, not_ok2, not_ok4] , axis = "index").reset_index(drop=True)
not_ok

In [ ]:
dodatek = pd.concat([not_ok, new_alright], axis = "index").reset_index(drop=True)
dodatek

In [ ]:
temp = dodatek.drop(columns=["_postprocessing", "is_ok"])
temp[temp.isna().all(axis=1)]

In [ ]:
dodatek.to_csv("parsed_4.csv", encoding = "utf-8")